# Laporan Analisis Bab

**Judul Buku:** Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
**Penulis:** Aurélien Géron
**Edisi:** Kedua, Diperbarui untuk TensorFlow 2

---

## Bab 15: Processing Sequences Using RNNs and CNNs

**I. Pendahuluan**
Bab 15 berfokus pada pemrosesan data sekuensial, sebuah kemampuan krusial dalam Machine Learning yang memungkinkan model memprediksi masa depan atau menganalisis data deret waktu seperti harga saham, suhu harian, atau metrik keuangan. Penulis memperkenalkan Recurrent Neural Networks (RNNs) sebagai kelas jaringan yang dapat menangani sekuens dengan panjang arbitrer, menjadikannya sangat berguna untuk aplikasi pemrosesan bahasa alami (NLP) seperti terjemahan otomatis atau *speech-to-text*. Bab ini akan menguraikan konsep fundamental RNN, cara melatihnya menggunakan *backpropagation through time*, dan mengatasi tantangan utama RNN: gradien yang tidak stabil dan memori jangka pendek yang terbatas. Selain RNN, bab ini juga mengeksplorasi penggunaan Convolutional Neural Networks (CNNs) 1D untuk pemrosesan sekuens yang sangat panjang, seperti pada arsitektur WaveNet.

**II. Recurrent Neurons and Layers**
Berbeda dengan *feedforward neural networks* di mana aktivasi mengalir satu arah, RNN memiliki koneksi yang menunjuk ke belakang, memungkinkan neuron untuk menerima input saat ini serta outputnya sendiri dari langkah waktu sebelumnya. Ini memberikan neuron bentuk "memori".

**Konsep-konsep Penting:**
* **Neuron Rekuren:** Pada setiap langkah waktu `t`, neuron rekuren menerima input `x(t)` dan outputnya sendiri dari langkah waktu sebelumnya `y(t-1)`.
* **Lapisan Rekuren:** Setiap neuron rekuren memiliki dua set bobot: satu untuk input `x(t)` (matriks bobot `Wx`) dan yang lain untuk output dari langkah waktu sebelumnya `y(t-1)` (matriks bobot `Wy`).
* **Unrolling the Network Through Time:** Representasi jaringan rekuren yang digambarkan sepanjang sumbu waktu, dengan satu neuron rekuren direpresentasikan per langkah waktu.
* **Memory Cells (Sel Memori):** Bagian dari jaringan saraf yang mempertahankan beberapa keadaan antar langkah waktu. Neuron rekuren tunggal atau lapisan neuron rekuren adalah sel dasar yang hanya dapat mempelajari pola pendek (biasanya sekitar 10 langkah).

**Input dan Output Sekuensial:**
* **Sequence-to-Sequence Network:** Menerima sekuens input dan menghasilkan sekuens output (misalnya, memprediksi deret waktu).
* **Sequence-to-Vector Network:** Menerima sekuens input, mengabaikan semua output kecuali yang terakhir (misalnya, analisis sentimen ulasan film).
* **Vector-to-Sequence Network:** Menerima input vektor yang sama berulang kali dan menghasilkan sekuens output (misalnya, pembuatan *caption* gambar).
* **Encoder-Decoder:** Menggabungkan *sequence-to-vector network* (encoder) diikuti oleh *vector-to-sequence network* (decoder) (misalnya, terjemahan bahasa).

**III. Pelatihan RNNs**
Pelatihan RNN melibatkan "membuka gulungannya" (unrolling) sepanjang waktu dan kemudian menggunakan *backpropagation* reguler, sebuah strategi yang disebut *backpropagation through time* (BPTT).

**Proses BPTT:**
1.  **Forward Pass:** Terjadi melalui jaringan yang belum digulung, menghitung output sekuens.
2.  **Evaluasi Cost Function:** Sekuens output dievaluasi menggunakan fungsi biaya `C(Y(0), Y(1), …Y(T))`.
3.  **Backward Pass:** Gradien fungsi biaya disebarkan mundur melalui jaringan yang belum digulung.
4.  **Parameter Update:** Parameter model diperbarui menggunakan gradien yang dihitung selama BPTT. Karena parameter yang sama (`W` dan `b`) digunakan di setiap langkah waktu, *backpropagation* akan menjumlahkannya di seluruh langkah waktu.

**Prakiraan Deret Waktu (Forecasting a Time Series):**
* **Data Deret Waktu:** Sekuens satu atau lebih nilai per langkah waktu (univariat atau multivariat).
* **Tugas Umum:** Memprediksi nilai masa depan (*forecasting*) atau mengisi nilai yang hilang (*imputation*).
* **Format Input:** Input umumnya direpresentasikan sebagai array 3D `[batch size, time steps, dimensionality]`.
* **Metrik Baseline:** Penting untuk memiliki metrik baseline (misalnya, *naive forecasting* atau model linier sederhana) untuk membandingkan kinerja model RNN.
* **Simple RNN dalam Keras:** Dapat dibangun dengan `keras.layers.SimpleRNN`. Secara *default*, lapisan ini hanya mengembalikan output terakhir.
* **Deep RNNs:** Menumpuk beberapa lapisan sel rekuren untuk membentuk RNN yang dalam. Untuk lapisan rekuren (kecuali yang terakhir jika hanya output terakhir yang penting), `return_sequences=True` harus diatur untuk memastikan output 3D diteruskan ke lapisan rekuren berikutnya.
* **Memprakirakan Beberapa Langkah Waktu ke Depan:**
    * **Satu per satu:** Memprediksi nilai berikutnya, menambahkannya ke input, dan menggunakannya untuk memprediksi nilai berikutnya lagi. Akurasi cenderung menurun seiring bertambahnya langkah waktu.
    * **Sekaligus:** Melatih RNN untuk memprediksi semua `N` nilai berikutnya secara bersamaan. Output lapisan terakhir memiliki `N` unit. Ini seringkali lebih baik dan lebih cepat daripada pendekatan satu per satu.
    * **Sequence-to-Sequence (Output per Langkah Waktu):** Melatih model untuk memprakirakan `N` nilai berikutnya pada setiap langkah waktu. Ini meningkatkan gradien kesalahan dan menstabilkan serta mempercepat pelatihan. `keras.layers.TimeDistributed` digunakan untuk menerapkan lapisan *dense* pada setiap langkah waktu.

**IV. Menangani Sekuens Panjang**
Melatih RNN pada sekuens yang sangat panjang menghadirkan dua masalah utama: gradien yang tidak stabil dan memori jangka pendek yang sangat terbatas.

**Melawan Masalah Gradien Tidak Stabil:**
* **Teknik Umum:** Inisialisasi parameter yang baik, *optimizer* yang lebih cepat, *dropout*, dll.
* **Fungsi Aktivasi:** Fungsi aktivasi yang tidak jenuh (misalnya ReLU) mungkin tidak banyak membantu dan bahkan dapat membuat RNN lebih tidak stabil. Fungsi aktivasi yang jenuh seperti *tanh* lebih disukai karena mencegah output meledak.
* **Gradient Clipping:** Mencegah gradien melebihi ambang batas tertentu selama *backpropagation*.
* **Batch Normalization (BN):** Kurang efisien pada RNN dibandingkan jaringan *feedforward*.
* **Layer Normalization:** Menormalisasi di sepanjang dimensi fitur (bukan dimensi *batch*). Ini dapat menghitung statistik yang diperlukan dengan cepat di setiap langkah waktu, secara independen untuk setiap instans. Berperilaku sama selama pelatihan dan pengujian. Lapisan `LNSimpleRNNCell` dapat dibuat secara kustom untuk mengimplementasikan *Layer Normalization*.

**Mengatasi Masalah Memori Jangka Pendek:**
* **LSTM Cells (Long Short-Term Memory):** Diperkenalkan pada tahun 1997. LSTM adalah sel memori jangka panjang yang sangat sukses dalam menangkap pola jangka panjang dalam deret waktu, teks panjang, rekaman audio, dll. Keras menyediakan lapisan `LSTM`.
    * **Arsitektur LSTM:** Keadaan sel terbagi menjadi dua vektor: keadaan jangka pendek (`h(t)`) dan keadaan jangka panjang (`c(t)`). Jaringan dapat belajar apa yang harus disimpan, dibuang, dan dibaca dari keadaan jangka panjang. Menggunakan *forget gate*, *input gate*, dan *output gate*.
    * **Peephole Connections:** Varian LSTM dengan koneksi tambahan yang memungkinkan pengontrol gerbang untuk "mengintip" keadaan jangka panjang.
* **GRU Cells (Gated Recurrent Unit):** Varian LSTM yang disederhanakan, seringkali berkinerja sama baiknya. Keras menyediakan lapisan `GRU`.
    * **Penyederhanaan GRU:** Menggabungkan dua vektor keadaan menjadi satu (`h(t)`). Pengontrol gerbang tunggal (`z(t)`) mengontrol *forget gate* dan *input gate*. Tidak ada *output gate*.

**Menggunakan Lapisan Konvolusional 1D untuk Memproses Sekuens:**
* Lapisan konvolusional 1D dapat digunakan untuk mempersingkat sekuens input (sub-sampel) sebelum diteruskan ke lapisan rekuren, membantu lapisan GRU/LSTM mendeteksi pola yang lebih panjang.
* **WaveNet:** Arsitektur yang diperkenalkan pada tahun 2016, menumpuk lapisan konvolusional 1D dengan tingkat dilasi yang berlipat ganda di setiap lapisan. Lapisan bawah mempelajari pola jangka pendek, sementara lapisan atas mempelajari pola jangka panjang. Sangat efisien untuk memproses sekuens yang sangat besar (puluhan ribu langkah waktu), seperti sampel audio.

**V. Kesimpulan**
Bab 15 mengupas tuntas tentang bagaimana RNN dan CNN digunakan untuk memproses data sekuensial. Ini mencakup dasar-dasar neuron dan lapisan rekuren, berbagai jenis arsitektur RNN untuk *forecasting* deret waktu, serta teknik-teknik canggih untuk mengatasi masalah gradien tidak stabil dan memori jangka pendek yang terbatas (seperti sel LSTM dan GRU). Selain itu, bab ini memperkenalkan bagaimana lapisan konvolusional 1D dapat diterapkan secara efektif untuk memproses sekuens yang sangat panjang, seperti pada arsitektur WaveNet yang revolusioner. Pembahasan ini memberikan landasan yang kuat untuk memahami pemrosesan bahasa alami di bab berikutnya.

# REPRODUCE CODE

## Forecasting a Time Series

In [1]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) # wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # + noise
    return series[..., np.newaxis].astype(np.float32)

In [2]:
import numpy as np

def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) # wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # + noise
    return series[..., np.newaxis].astype(np.float32)

n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

## Baseline Metrics

In [3]:
!pip install tensorflow

In [4]:
import numpy as np
import tensorflow as tf # Import tensorflow
from tensorflow import keras # Import keras from tensorflow

def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) # wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # + noise
    return series[..., np.newaxis].astype(np.float32)

n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

y_pred = X_valid[:, -1]

# Use tf.keras.metrics.MeanSquaredError by creating an instance
# and calling its result() method after updating it.
mse_metric = tf.keras.metrics.MeanSquaredError()
mse_metric.update_state(y_valid, y_pred)
print(mse_metric.result().numpy()) # Print the result, converting the tensor to a numpy value

0.020900993


In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## Implementing a Simple RNN

In [6]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## Deep RNNs

In [7]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

In [8]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

## Forecasting Several Time Steps Ahead

In [9]:
series = generate_time_series(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new
for step_ahead in range(10):
    y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)

Y_pred = X[:, n_steps:]

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


In [10]:
series = generate_time_series(10000, n_steps + 10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

In [11]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

In [12]:
Y_pred = model.predict(X_new)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 921ms/step


In [13]:
Y = np.empty((10000, n_steps, 10)) # each target is a sequence of 10D vectors
for step_ahead in range(1, 10 + 1):
    Y[:, :, step_ahead - 1] = series[:, step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

In [14]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [15]:
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

# Change lr=0.01 to learning_rate=0.01
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss="mse", optimizer=optimizer, metrics=[last_time_step_mse])

## Fighting the Unstable Gradients Problem

In [16]:
class LNSimpleRNNCell(keras.layers.Layer):
    def __init__(self, units, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = keras.layers.SimpleRNNCell(units,
                                                          activation=None)
        self.layer_norm = keras.layers.LayerNormalization()
        self.activation = keras.activations.get(activation)
    def call(self, inputs, states):
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]

In [17]:
model = keras.models.Sequential([
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True,
                     input_shape=[None, 1]),
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'ln_simple_rnn_cell', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'ln_simple_rnn_cell_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


## Tackling the Short-Term Memory Problem

In [18]:
model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [19]:
model = keras.models.Sequential([
    keras.layers.RNN(keras.layers.LSTMCell(20), return_sequences=True,
                     input_shape=[None, 1]),
    keras.layers.RNN(keras.layers.LSTMCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [20]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Define the time series generation function
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) # wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # + noise
    return series[..., np.newaxis].astype(np.float32)

# Generate data
n_steps = 50
# Generate a bit longer series for target, ensuring we have enough steps
# for targets after the Conv1D output sequence.
# Conv1D output length = 24. Last Conv1D output index = 23.
# Corresponds to input index 2*23 = 46.
# Window is 46 to 49. Target starts at 50 and needs 10 steps (50 to 59).
# So, original series needs length n_steps + kernel_size + (target_steps - 1)
# 50 + 4 + (10 - 1) = 50 + 4 + 9 = 63
# Let's generate slightly more to be safe.
series_length = n_steps + 10 + 10 # n_steps + target_length + some buffer
series = generate_time_series(10000, series_length)
X_train = series[:7000, :n_steps]
X_valid = series[7000:9000, :n_steps]
X_test = series[9000:, :n_steps]

# Define the Conv1D layer parameters
kernel_size = 4
strides = 2
conv1d_output_steps = (n_steps - kernel_size) // strides + 1 # Calculate Conv1D output length

# Prepare target data to match the Conv1D output sequence length
# Y will have shape (batch_size, conv1d_output_steps, 10)
Y = np.empty((10000, conv1d_output_steps, 10))
for i in range(conv1d_output_steps):
    # The i-th output of Conv1D (with stride 2, kernel 4) corresponds to
    # input slice from 2*i to 2*i + 3.
    # We want to predict the next 10 steps *after* this slice.
    # These steps are from index (2*i) + 4 to (2*i) + 4 + 10 - 1.
    start_target_index = (2 * i) + kernel_size
    end_target_index = start_target_index + 10
    Y[:, i, :] = series[:, start_target_index:end_target_index, 0]


Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

# Define the custom metric
def last_time_step_mse(Y_true, Y_pred):
    # Ensure Y_true and Y_pred have the same rank for slicing
    # Y_true[:, -1] takes the last time step of the true targets (shape: (batch_size, 10))
    # Y_pred[:, -1] takes the last time step of the predicted outputs (shape: (batch_size, 10))
    # Use tf.math.square and tf.reduce_mean for calculating MSE on the last time step
    return tf.reduce_mean(tf.math.square(Y_true[:, -1] - Y_pred[:, -1]))

# Define the model
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=kernel_size, strides=strides, padding="valid",
                         input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

# Compile the model
model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])

# Train the model
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - last_time_step_mse: 0.0915 - loss: 0.0971 - val_last_time_step_mse: 0.0347 - val_loss: 0.0435
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - last_time_step_mse: 0.0331 - loss: 0.0397 - val_last_time_step_mse: 0.0227 - val_loss: 0.0313
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - last_time_step_mse: 0.0219 - loss: 0.0302 - val_last_time_step_mse: 0.0170 - val_loss: 0.0273
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - last_time_step_mse: 0.0162 - loss: 0.0265 - val_last_time_step_mse: 0.0135 - val_loss: 0.0246
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - last_time_step_mse: 0.0138 - loss: 0.0246 - val_last_time_step_mse: 0.0120 - val_loss: 0.0232
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - last_time_step_mse: 0.0122 - loss: 0.0234 - val_last_time_step_mse: 0.0113 - val_loss: 0.0223
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - last_time_step_mse: 0.0114 - loss: 0.0223 - val_last_ti

## Wavenet

In [22]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Define the time series generation function
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) # wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # + noise
    return series[..., np.newaxis].astype(np.float32)

# Generate data
n_steps = 50
# Need enough steps in the original series to provide targets for each of the
# n_steps (50) input steps, predicting 10 steps ahead.
# The target for input step i is series[i+1:i+11].
# For the last input step (49), the target is series[50:60].
# So, the original series needs at least 50 + 10 = 60 steps.
series_length = n_steps + 10
series = generate_time_series(10000, series_length)

X_train = series[:7000, :n_steps]
X_valid = series[7000:9000, :n_steps]
X_test = series[9000:, :n_steps]

# Prepare target data for the WaveNet model with causal padding.
# The model outputs a sequence of length n_steps (50).
# For each input step i (from 0 to 49), the model predicts the next 10 steps.
# So, the target Y will have shape (batch_size, n_steps, 10).
# Y[batch, i, :] should be the series values from series[batch, i+1] to series[batch, i+10].
Y = np.empty((10000, n_steps, 10), dtype=np.float32)
for step in range(n_steps):
    # The target for input at time `step` is the sequence from `step + 1` to `step + 10`.
    Y[:, step, :] = series[:, step + 1 : step + 1 + 10, 0]

Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]


# Define the custom metric
def last_time_step_mse(Y_true, Y_pred):
    # Ensure Y_true and Y_pred have the same rank for slicing
    # Y_true[:, -1] takes the last time step of the true targets (shape: (batch_size, 10))
    # Y_pred[:, -1] takes the last time step of the predicted outputs (shape: (batch_size, 10))
    # Use tf.math.square and tf.reduce_mean for calculating MSE on the last time step
    return tf.reduce_mean(tf.math.square(Y_true[:, -1] - Y_pred[:, -1]))

# Define the model (this part remains the same)
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None, 1]))
for rate in (1, 2, 4, 8) * 2:
    model.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding="causal",
                                  activation="relu", dilation_rate=rate))
model.add(keras.layers.Conv1D(filters=10, kernel_size=1))

# Compile the model
model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])

# Train the model
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - last_time_step_mse: 0.0899 - loss: 0.0976 - val_last_time_step_mse: 0.0226 - val_loss: 0.0357
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - last_time_step_mse: 0.0202 - loss: 0.0326 - val_last_time_step_mse: 0.0171 - val_loss: 0.0294
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - last_time_step_mse: 0.0163 - loss: 0.0284 - val_last_time_step_mse: 0.0148 - val_loss: 0.0272
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - last_time_step_mse: 0.0143 - loss: 0.0263 - val_last_time_step_mse: 0.0131 - val_loss: 0.0259
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - last_time_step_mse: 0.0134 - loss: 0.0251 - val_last_time_step_mse: 0.0125 - val_loss: 0.0247
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - last_time_step_mse: 0.0124 - loss: 0.0241 - val_last_time_step_mse: 0.0121 - val_loss: 0.0242
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - last_time_step_mse: 0.0116 - loss: 0.0233 - val_last_tim